# [EXPERIMENTAL] SupplySeer Geopolitical API

This tutorial shows a surface level demonstration of the Geopolitical API (you can use the API locally but not over internet yet). <br>
It is using the GDELT Project API to grab global events from news media over the worlds nations in realtime.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [24]:
from supplyseer.experimental.exotic_supplychain.geopolitical import (create_current_conflict_scenario, 
                                                                     EnhancedGeopoliticalRiskModule, 
                                                                     ConflictZone, ConflictType, ConflictImpactReport, 
                                                                     RiskCategory, RiskLevel, RiskZone, 
                                                                     SanctionImpact, GeopoliticalEvent, GeopoliticalRiskModule)

from supplyseer.experimental.exotic_supplychain.gdeltmonitor import *

### Quick Conflict Scenario
This is a predefined conflict scenario that you can just grab with the function below.

In [6]:
risk_module = create_current_conflict_scenario()

Added event: Ukraine-Russia Conflict in Ukraine with risk level critical
Added event: Taiwan Strait Tensions in Taiwan with risk level high
Added event: Red Sea Shipping Crisis in Red Sea with risk level severe


It also comes with different methods that helps a Supply Chain Analyst

In [5]:
# Test commodity risk analysis
semiconductor_risk = risk_module.analyze_commodity_risk("semiconductors")
print(f"Semiconductor Risk Level: {semiconductor_risk.risk_level}")
print(f"Affected by conflicts: {semiconductor_risk.affected_by_conflicts}")
print(f"Alternative sources: {semiconductor_risk.alternative_sources}")

# Generate impact report
impact_report = risk_module.generate_conflict_impact_report()
for report in impact_report:
    print(f"\nConflict: {report.conflict_name}")
    print(f"Severity: {report.severity}")
    print(f"Lead Time Impact: {report.lead_time_impact}")
    print(f"Cost Impact: {report.cost_impact}")

Semiconductor Risk Level: RiskLevel.HIGH
Affected by conflicts: [{'conflict': 'Taiwan Strait Tensions', 'severity': <RiskLevel.HIGH: 'high'>, 'affected_regions': ['Taiwan', 'South China Sea'], 'sanctions': []}]
Alternative sources: []

Conflict: Ukraine-Russia Conflict
Severity: critical
Lead Time Impact: 500.0%
Cost Impact: 650.0%

Conflict: Taiwan Strait Tensions
Severity: high
Lead Time Impact: 95.0%
Cost Impact: 110.0%

Conflict: Red Sea Shipping Crisis
Severity: severe
Lead Time Impact: 300.0%
Cost Impact: 400.0%


In [8]:
risk_module.calculate_route_risk(
    origin="Shanghai",
    destination="Rotterdam",
    route_waypoints=[
        "Singapore",
        "Suez Canal",
        "Mediterranean"
    ],
    date=datetime(2024, 2, 1)
)

RouteRiskAssessment(risk_score=1.0, delay_factor=1.0, cost_factor=1.0, impacted_segments=[], risk_level=<RiskLevel.LOW: 'low'>)

### Designing your own Geopolitical Risk model  
Your model should be empty from the beginning when instantiating it

In [43]:
risk_module = EnhancedGeopoliticalRiskModule()
risk_module.active_events, risk_module.sanctions_registry, risk_module.conflict_zones

([], {}, [])

**Explore the parts of the module**

In [23]:
# Conflict types

ConflictType.ARMED_CONFLICT
ConflictType.CIVIL_UNREST
ConflictType.ECONOMIC_WARFARE
ConflictType.TERRITORIAL_DISPUTE 
ConflictType.TRADE_SANCTIONS

# Risk categories
RiskCategory.ECONOMIC
RiskCategory.POLITICAL
RiskCategory.REGULATORY
RiskCategory.SECURITY
RiskCategory.TRADE

# Risk level
RiskLevel.CRITICAL
RiskLevel.HIGH
RiskLevel.MEDIUM
RiskLevel.LOW
RiskLevel.SEVERE


<RiskLevel.SEVERE: 'severe'>

In [28]:
# Sanctions
SanctionImpact(
    restricted_items=["technology", "industrial equipment", "luxury goods"],    # Items restricted because of the conflict
    alternative_sources=["EU", "USA", "Japan"],         # Other sources to get the items from
    cost_increase_factor=1.8,           # Increase in costs by a certain factor
    lead_time_increase=45,              # Lead time is affected by the conflict
    reliability_decrease=0.4            # Decrease in reliability, inactive as of now
)

SanctionImpact(restricted_items=['technology', 'industrial equipment', 'luxury goods'], alternative_sources=['EU', 'USA', 'Japan'], cost_increase_factor=1.8, lead_time_increase=45, reliability_decrease=0.4)

#### Add your own conflict zones

In [44]:
ukraine_conflict = ConflictZone(
    name="Ukraine-Russia Conflict", # Name the conflict
    conflict_type=ConflictType.ARMED_CONFLICT,  # Define the conflict type
    affected_regions=["Ukraine", "Russia", "Belarus"],  # Add regions affected by the conflict
    start_date=datetime(2022, 2, 24),       # Starting date
    severity=RiskLevel.CRITICAL,            # Define the risk level
    displaced_population=8000000,           # Population displaced from the affected regions
    strategic_commodities=["grain", "metals", "energy", "fertilizers"], # Commodities under risk
    sanctions=[             # Define the sanctions and their impact
        SanctionImpact(
            restricted_items=["technology", "industrial equipment", "grain", "luxury goods"],
            alternative_sources=["EU", "USA", "Japan"],
            cost_increase_factor=1.8,
            lead_time_increase=45,
            reliability_decrease=0.4
        )
    ]
)

In [45]:
ukraine_conflict.affected_regions, ukraine_conflict.conflict_type, ukraine_conflict.sanctions

(['Ukraine', 'Russia', 'Belarus'],
 <ConflictType.ARMED_CONFLICT: 'armed_conflict'>,
 [SanctionImpact(restricted_items=['technology', 'industrial equipment', 'grain', 'luxury goods'], alternative_sources=['EU', 'USA', 'Japan'], cost_increase_factor=1.8, lead_time_increase=45, reliability_decrease=0.4)])

In [46]:
taiwan_tension = ConflictZone(
    name="Taiwan Strait Tensions",
    conflict_type=ConflictType.TERRITORIAL_DISPUTE,
    affected_regions=["Taiwan", "South China Sea"],
    start_date=datetime(2024, 1, 1),
    severity=RiskLevel.HIGH,
    strategic_commodities=["semiconductors", "electronics", "computer chips"],
    sanctions=[
        SanctionImpact(
            restricted_items=["advanced semiconductors", "chip manufacturing equipment"],
            alternative_sources=["South Korea", "Japan", "USA"],
            cost_increase_factor=1.5,
            lead_time_increase=30,
            reliability_decrease=0.3
        )
    ]
)

risk_module.add_conflict_zone(ukraine_conflict)
risk_module.add_conflict_zone(taiwan_tension)

Added event: Ukraine-Russia Conflict in Ukraine with risk level critical
Added event: Taiwan Strait Tensions in Taiwan with risk level high


### Generate an impact report

**Alternative sources for sanctioned commodities** <br>
You are Sweden and you buy grains from Russia, Belarus, and Ukraine before the conflict. <br>
Suddenly, you don't know what to do because grains are under sanctions but you need to provide alternative sources.
<br>
<br>
At the same time, you need to know the impact on lead time and costs.

In [65]:
# Test commodity risk analysis
semiconductor_risk = risk_module.analyze_commodity_risk("grain")
print(f"Semiconductor Risk Level: {semiconductor_risk.risk_level}")
print(f"Affected by conflicts:\n{semiconductor_risk.affected_by_conflicts}")
print(f"Alternative sources: {semiconductor_risk.alternative_sources}")

# Generate impact report
impact_report = risk_module.generate_conflict_impact_report()
for report in impact_report:
    print(f"\nConflict: {report.conflict_name}")
    print(f"Severity: {report.severity}")
    print(f"Lead Time Impact: {report.lead_time_impact}")
    print(f"Cost Impact: {report.cost_impact}")

Semiconductor Risk Level: RiskLevel.CRITICAL
Affected by conflicts:
[{'conflict': 'Ukraine-Russia Conflict', 'severity': <RiskLevel.CRITICAL: 'critical'>, 'affected_regions': ['Ukraine', 'Russia', 'Belarus'], 'sanctions': [['technology', 'industrial equipment', 'grain', 'luxury goods']]}]
Alternative sources: ['USA', 'Japan', 'EU']

Conflict: Ukraine-Russia Conflict
Severity: critical
Lead Time Impact: 500.0%
Cost Impact: 650.0%

Conflict: Taiwan Strait Tensions
Severity: high
Lead Time Impact: 95.0%
Cost Impact: 110.0%


## GDELT Project API - Global Supply Chain Risk Monitor

This part shows the GDELT Monitor module that lets you query in realtime for media coverage globally. <br>
We will also demonstrate what to do with the data we receive, namely natural language understanding by using <br>
sentiment analysis models from HuggingFace.


In [66]:
gdelt_monitor = SupplyChainGDELTMonitor()

In [67]:
# This is the preferred way in Jupyter notebooks
async def get_risks():
    async with SupplyChainGDELTMonitor() as gdelt_monitor:
        risks = await gdelt_monitor.monitor_region_risks("sweden", timespan="7d")
        return risks

In [68]:
# Then run it with await
risks = await get_risks()
pd.json_normalize(risks)

,supply_disruption,geopolitical,infrastructure,trade_restrictions,natural_disasters
0,[{'title': 'Dutch health minister admits that ...,[{'title': 'Germany AfD Urges UN to Investigat...,[],[],[]


In [69]:
output = pd.json_normalize(risks, record_path="geopolitical")

In [70]:
output.head(20)

,title,url,source_country,tone,date,themes
0,Germany AfD Urges UN to Investigate Nord Strea...,https://sputnikglobe.com/20241115/german-afd-u...,,0.0,20241115T121500Z,[]
1,Canada opens path to permanent residency for P...,https://www.piquenewsmagazine.com/must-reads/c...,Canada,0.0,20241115T061500Z,[]
2,Canada opens path to permanent residency for P...,https://www.delta-optimist.com/highlights/cana...,Canada,0.0,20241115T044500Z,[]
3,Canada opens path to permanent residency for P...,https://www.squamishchief.com/highlights/canad...,Canada,0.0,20241115T043000Z,[]
4,Canada opens path to permanent residency for P...,https://www.burnabynow.com/highlights/canada-o...,Canada,0.0,20241115T041500Z,[]
5,Canada opens path to permanent residency for P...,https://www.bowenislandundercurrent.com/highli...,Canada,0.0,20241115T040000Z,[]
6,The Day of the Jackal Recap Premiere Episode 1...,https://tvline.com/recaps/the-day-of-the-jacka...,United States,0.0,20241115T034500Z,[]
7,Canada opens path to permanent residency for P...,https://www.tricitynews.com/highlights/canada-...,Canada,0.0,20241115T033000Z,[]
8,Canada opens path to permanent residency for P...,https://www.richmond-news.com/highlights/canad...,Canada,0.0,20241115T030000Z,[]
9,Norway lowers terror threat level as Iran thre...,https://thefrontierpost.com/norway-lowers-terr...,Pakistan,0.0,20241114T183000Z,[]


### [OPTIONAL] Natural Language Understanding


In [ ]:
# !pip install transformers # Uncomment this
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", device="cuda") # Remove device argument if you dont have a GPU

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


**Put titles and origins in list format**

In [72]:
titles = output.title.to_list()
origins = output["source_country"].to_list()

### Sentiment Analysis

In [77]:
for title, origin in zip(titles[35:42], origins[35:42]):
    sentiment = sentiment_pipeline(title)[0]["label"].lower()
    score = sentiment_pipeline(title)[0]["score"]*100
    score = round(score, 2)

    print(f"Origin country: {origin}")
    print(f"Title: '{title}' \n{score}% {sentiment}")
    print("---"*10)

Origin country: United States
Title: 'Trump and the  sick man  of Europe' 
97.97% positive
------------------------------
Origin country: Israel
Title: 'At the table with Jeremy Berkovits : The face of American Colony Hotel - Israel News' 
99.78% positive
------------------------------
Origin country: Bangladesh
Title: 'Trump second term and the future of Turkish - US relations' 
99.64% positive
------------------------------
Origin country: Pakistan
Title: 'Trump election win casts scrutiny on Biden - Harris foreign policy legacy' 
86.12% negative
------------------------------
Origin country: 
Title: 'German President Loses Cool Over Criticism of His Stance on NATO , Nord Stream' 
99.97% negative
------------------------------
Origin country: Turkey
Title: 'Biden foreign policy legacy faces scrutiny after Trump 2024 victory' 
98.18% negative
------------------------------
Origin country: United States
Title: 'Sweden Sentences Quran Burning Activist Rasmus Paludan to Jail' 
97.97% neg